# Computación en Arrays NumPy: Funciones universales

Hasta ahora, hemos discutido algunos de los aspectos básicos de NumPy; en las próximas secciones, nos sumergiremos en las razones por las que NumPy es tan importante en el mundo de la ciencia de datos de Python.
A saber, proporciona una interfaz fácil y flexible para la computación optimizada con matrices de datos.

La computación en arrays de NumPy puede ser muy rápida, o puede ser muy lenta.
La clave para hacerla rápida es usar operaciones *vectorizadas*, generalmente implementadas a través de las *funciones universales* de NumPy (ufuncs).
Esta sección motiva la necesidad de las ufuncs de NumPy, que pueden usarse para hacer cálculos repetidos sobre elementos del array mucho más eficientes.
A continuación, se presentan muchas de las ufunciones aritméticas más comunes y útiles disponibles en el paquete NumPy.

## La lentitud de los bucles

La implementación por defecto de Python (conocida como CPython) realiza algunas operaciones muy lentamente.
Esto se debe en parte a la naturaleza dinámica e interpretada del lenguaje: el hecho de que los tipos son flexibles, de modo que las secuencias de operaciones no pueden ser compiladas a código máquina eficiente como en lenguajes como C y Fortran.
Recientemente ha habido varios intentos de abordar esta debilidad: ejemplos conocidos son el proyecto [PyPy](http://pypy.org/), una implementación compilada justo a tiempo de Python; el proyecto [Cython](http://cython.org), que convierte el código Python en código C compilable; y el proyecto [Numba](http://numba.pydata.org/), que convierte fragmentos de código Python en bytecode LLVM rápido.
Cada uno de ellos tiene sus puntos fuertes y débiles, pero se puede decir que ninguno de los tres enfoques ha superado todavía el alcance y la popularidad del motor estándar de CPython.

La relativa lentitud de Python se manifiesta generalmente en situaciones en las que se repiten muchas operaciones pequeñas - por ejemplo, el bucle sobre matrices para operar en cada elemento.
Por ejemplo, imaginemos que tenemos una matriz de valores y queremos calcular el recíproco de cada uno.
Un enfoque directo podría ser el siguiente:

In [ ]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

Esta implementación probablemente resulte bastante natural para alguien que tenga, por ejemplo, conocimientos de C o Java.
Pero si medimos el tiempo de ejecución de este código para una entrada grande, vemos que esta operación es muy lenta, ¡quizás sorprendentemente!
Vamos a comparar esto con la magia ``%timeit`` de IPython (discutido en [Código de perfiles y tiempos](01.07-Timing-and-Profiling.ipynb)):

In [ ]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1 loop, best of 3: 2.91 s per loop


El cálculo de este millón de operaciones y el almacenamiento del resultado tardan varios segundos.
Cuando incluso los teléfonos móviles tienen velocidades de procesamiento que se miden en Giga-FLOPS (es decir, miles de millones de operaciones numéricas por segundo), esto parece casi absurdamente lento.
Resulta que el cuello de botella aquí no son las operaciones en sí, sino la comprobación de tipos y los envíos de funciones que CPython debe hacer en cada ciclo del bucle.
Cada vez que se calcula el recíproco, Python examina primero el tipo del objeto y hace una búsqueda dinámica de la función correcta a utilizar para ese tipo.
Si estuviéramos trabajando en código compilado, esta especificación de tipo sería conocida antes de que el código se ejecute y el resultado podría ser calculado mucho más eficientemente.

## Presentando UFuncs

Para muchos tipos de operaciones, NumPy proporciona una interfaz conveniente para este tipo de rutina compilada y tipada estáticamente. Esto se conoce como una operación *vectorizada*.
Esto se puede lograr simplemente realizando una operación en el array, que luego se aplicará a cada elemento.
Este enfoque vectorizado está diseñado para empujar el bucle en la capa compilada que subyace a NumPy, lo que lleva a una ejecución mucho más rápida.

Compara los resultados de los dos siguientes:

In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

[ 0.16666667  1.          0.25        0.25        0.125     ]
[ 0.16666667  1.          0.25        0.25        0.125     ]


Si observamos el tiempo de ejecución de nuestro gran array, vemos que se completa órdenes de magnitud más rápido que el bucle de Python:

In [ ]:
%timeit (1.0 / big_array)

100 loops, best of 3: 4.6 ms per loop


Las operaciones vectoriales en NumPy se implementan a través de *ufuncs*, cuyo propósito principal es ejecutar rápidamente operaciones repetidas sobre valores en arrays de NumPy.
Los Ufuncs son extremadamente flexibles - antes vimos una operación entre un escalar y un array, pero también podemos operar entre dos arrays:

In [ ]:
np.arange(5) / np.arange(1, 6)

array([ 0.        ,  0.5       ,  0.66666667,  0.75      ,  0.8       ])

Además, las operaciones ufunc no se limitan a las matrices unidimensionales, sino que también pueden actuar sobre matrices multidimensionales:

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

Los cálculos que utilizan la vectorización a través de ufuncs son casi siempre más eficientes que su contraparte implementada utilizando bucles de Python, especialmente cuando las matrices crecen en tamaño.
Cada vez que veas un bucle de este tipo en un script de Python, deberías considerar si puede ser reemplazado por una expresión vectorizada.

## Explorando los UFuncs de NumPy

Los Ufuncs existen en dos sabores: *unary ufuncs*, que operan en una sola entrada, y *binary ufuncs*, que operan en dos entradas.
Aquí veremos ejemplos de ambos tipos de funciones.

### Aritmética de matrices

Los ufuncs de NumPy se sienten muy naturales de usar porque hacen uso de los operadores aritméticos nativos de Python.
La suma, la resta, la multiplicación y la división estándar pueden ser utilizadas:

In [ ]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [ 0.   0.5  1.   1.5]
x // 2 = [0 0 1 1]


También hay un ufunc unario para la negación, y un operador ``**`` para la exponenciación, y un operador ``%`` para el módulo:

In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


Además, se pueden encadenar como se quiera y se respeta el orden estándar de las operaciones:

In [ ]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

Cada una de estas operaciones aritméticas son simplemente envolturas convenientes alrededor de funciones específicas construidas en NumPy; por ejemplo, el operador ``+`` es una envoltura para la función ``add``:

In [ ]:
np.add(x, 2)

array([2, 3, 4, 5])

La siguiente tabla lista los operadores aritméticos implementados en NumPy:

| Operador 	    | Ufunc equivalente   | Descripción                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Adición (p.ej., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Sustracción (p.ej., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Negación unaria (p.ej., ``-2``)         |
|``*``          |``np.multiply``      |Multiplicación (p.ej., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |División (p.ej., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor división (p.ej., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponenciación (p.ej., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulo/Resto (p.ej., ``9 % 4 = 1``)|

Además, existen operadores booleanos/de bits; los exploraremos en [Comparaciones, máscaras y lógica booleana](02.06-Boolean-Arrays-and-Masks.ipynb).

### Valor absoluto

Al igual que NumPy entiende los operadores aritméticos incorporados de Python, también entiende la función de valor absoluto incorporada de Python:

In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

La correspondiente ufunc de NumPy es ``np.absolute``, que también está disponible bajo el alias ``np.abs``:

In [ ]:
np.absolute(x)

In [ ]:
np.abs(x)

Esta ufunc también puede manejar datos complejos, en los que el valor absoluto devuelve la magnitud:

In [ ]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

### Funciones trigonométricas

NumPy proporciona un gran número de ufunciones útiles, y algunas de las más útiles para el científico de datos son las funciones trigonométricas.
Empezaremos por definir un array de ángulos:

In [ ]:
theta = np.linspace(0, np.pi, 3)

Ahora podemos calcular algunas funciones trigonométricas sobre estos valores:

In [ ]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [ 0.          1.57079633  3.14159265]
sin(theta) =  [  0.00000000e+00   1.00000000e+00   1.22464680e-16]
cos(theta) =  [  1.00000000e+00   6.12323400e-17  -1.00000000e+00]
tan(theta) =  [  0.00000000e+00   1.63312394e+16  -1.22464680e-16]


Los valores se calculan con una precisión de máquina, por lo que los valores que deberían ser cero no siempre llegan exactamente a cero.
También están disponibles las funciones trigonométricas inversas:

In [ ]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [ 3.14159265  1.57079633  0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponentes y logaritmos

Otro tipo de operación común disponible en un ufunc de NumPy son los exponenciales:

In [ ]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [  2.71828183   7.3890561   20.08553692]
2^x   = [ 2.  4.  8.]
3^x   = [ 3  9 27]


La inversa de los exponenciales, los logaritmos, también están disponibles.
El básico ``np.log`` da el logaritmo natural; si prefieres calcular el logaritmo de base 2 o el de base 10, también están disponibles:

In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [ 0.          0.69314718  1.38629436  2.30258509]
log2(x)  = [ 0.          1.          2.          3.32192809]
log10(x) = [ 0.          0.30103     0.60205999  1.        ]


También hay algunas versiones especializadas que son útiles para mantener la precisión con entradas muy pequeñas:

In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [ 0.          0.0010005   0.01005017  0.10517092]
log(1 + x) = [ 0.          0.0009995   0.00995033  0.09531018]


Cuando ``x`` es muy pequeño, estas funciones dan valores más precisos que si se utiliza el ``np.log`` o el ``np.exp`` en bruto.

### Ufunciones especializadas

NumPy tiene muchas más ufunciones disponibles, incluyendo funciones trigonométricas hiperbólicas, aritmética a nivel de bits, operadores de comparación, conversiones de radianes a grados, redondeo y residuos, y mucho más.
Un vistazo a la documentación de NumPy revela muchas funciones interesantes.

Otra excelente fuente de ufunciones más especializadas y oscuras es el submódulo ``scipy.special``.
Si quieres calcular alguna función matemática oscura sobre tus datos, lo más probable es que esté implementada en ``scipy.special``.
Hay demasiadas funciones para enumerarlas todas, pero el siguiente fragmento muestra un par de ellas que podrían surgir en un contexto estadístico:

In [ ]:
from scipy import special

In [ ]:
# Funciones gamma (factoriales generalizados) y funciones afines
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [  1.00000000e+00   2.40000000e+01   3.62880000e+05]
ln|gamma(x)| = [  0.           3.17805383  12.80182748]
beta(x, 2)   = [ 0.5         0.03333333  0.00909091]


In [ ]:
# Función de error (integral de Gauss)
# su complemento, y su inversa
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [ 0.          0.32862676  0.67780119  0.84270079]
erfc(x) = [ 1.          0.67137324  0.32219881  0.15729921]
erfinv(x) = [ 0.          0.27246271  0.73286908         inf]


Hay muchos, muchos más ufuncs disponibles tanto en NumPy como en ``scipy.special``.
Dado que la documentación de estos paquetes está disponible en línea, una búsqueda en la web a lo largo de las líneas de "gamma function python" generalmente encontrará la información relevante.

## Características avanzadas de Ufunc

Muchos usuarios de NumPy hacen uso de ufuncs sin llegar a aprender todo su conjunto de características.
Vamos a describir algunas características especializadas de ufuncs aquí.

### Especificación de la salida

Para cálculos grandes, a veces es útil poder especificar el array donde se almacenará el resultado del cálculo.
En lugar de crear un array temporal, se puede utilizar para escribir los resultados de los cálculos directamente en la ubicación de memoria donde se desea que estén.
Para todos los ufuncs, esto se puede hacer utilizando el argumento ``out`` de la función:

In [ ]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[  0.  10.  20.  30.  40.]


Esto se puede utilizar incluso con vistas de matrices. Por ejemplo, podemos escribir los resultados de un cálculo en cada uno de los elementos de un array especificado:

In [ ]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[  1.   0.   2.   0.   4.   0.   8.   0.  16.   0.]


Si en lugar de eso hubiéramos escrito ``y[::2] = 2 ** x``, se habría creado una matriz temporal para mantener los resultados de ``2 ** x``, seguida de una segunda operación para copiar esos valores en la matriz ``y``.
Esto no supone una gran diferencia para un cálculo tan pequeño, pero para matrices muy grandes el ahorro de memoria por el uso cuidadoso del argumento ``out`` puede ser significativo.

### Agregados

Para los ufuncs binarios, hay algunos agregados interesantes que se pueden calcular directamente desde el objeto.
Por ejemplo, si queremos *reducir* un array con una operación determinada, podemos utilizar el método ``reduce`` de cualquier ufunc.
Una reducción aplica repetidamente una operación determinada a los elementos de un array hasta que sólo queda un único resultado.

Por ejemplo, llamar a ``reduce`` en la ufunc ``add`` devuelve la suma de todos los elementos del array:

In [ ]:
x = np.arange(1, 6)
np.add.reduce(x)

15

Del mismo modo, llamar a ``reduce`` en la ufunc ``multiplicar`` da como resultado el producto de todos los elementos del array:

In [ ]:
np.multiply.reduce(x)

120

Si queremos almacenar todos los resultados intermedios del cálculo, podemos utilizar ``acumular``:

In [ ]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [ ]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

Ten en cuenta que para estos casos particulares, hay funciones NumPy dedicadas a calcular los resultados (``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``), que exploraremos en [Agregaciones: Mín, Máx, y todo lo que hay entre medias](02.04-Computación-en-arrays-agregados.ipynb).

### Productos externos

Finalmente, cualquier ufunc puede calcular la salida de todos los pares de dos entradas diferentes utilizando el método ``outer``.
Esto te permite, en una línea, hacer cosas como crear una tabla de multiplicación:

In [ ]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

Los métodos ``ufunc.at`` y ``ufunc.reduceat``, que exploraremos en [Fancy Indexing](02.07-Fancy-Indexing.ipynb), son también muy útiles.

Otra característica extremadamente útil de ufuncs es la capacidad de operar entre arrays de diferentes tamaños y formas, un conjunto de operaciones conocido como *broadcasting*.
Este tema es lo suficientemente importante como para que le dediquemos una sección entera (ver [Computación en matrices: Broadcasting](02.05-Computación-en-matrices-broadcasting.ipynb)).

## Ufuncs: Aprender más

Puede encontrar más información sobre las funciones universales (incluyendo la lista completa de funciones disponibles) en los sitios web de documentación de [NumPy](http://www.numpy.org) y [SciPy](http://www.scipy.org).

Recuerde que también puede acceder a la información directamente desde IPython importando los paquetes y utilizando la funcionalidad de completado de pestañas y ayuda (``?``) de IPython, como se describe en [Ayuda y documentación en IPython](01.01-Ayuda-Y-Documentación.ipynb).